## Análisis Exploratorio de los Atributos

### Creación de nuevas dimensiones

A partir de la exploración de los atributos, las nuevas columnas que agregamos en nuestro trabajo fueron: 


##### Combinación de atributos existentes

* price_diff: Esta columna calcula la diferencia de precio absoluta entre las columnas "price" y "original_price".

* discount: Calcula el porcentaje de descuento para cada artículo comparando la diferencia entre "original_price" y "price" sobre "original_price".

* price_ratio: Calcula la proporción de precio dividiendo "price" sobre "original_price".

* is_discount: Una columna binaria que indica si hay un descuento (1) o no (0) basándose en el valor de la columna "discount".

* title_length: Longitud en caracteres de la columna "title" para cada artículo.

* title_word_count: Cantidad de palabras en la columna "title" dividiendo el texto por espacios.

* title_length_word_count: Calcula la proporción entre "title_length" y "title_word_count", lo que puede ayudar a identificar títulos con diferentes niveles de verbosidad.

* domain_dominance: Calcula una medida de cuán dominante es el artículo dentro de su dominio dividiendo "sold_quantity" entre "qty_items_dom".

* is_pdp_tvi: Calcula la proporción de "is_pdp" entre "total_visits_item", representando la proporción de visitas de artículo que resultan en una vista de Página de Detalles del Producto (PDP).

* is_pdp_tvs: Calcula la proporción de "is_pdp" entre "total_visits_seller", representando la proporción de visitas a vendedores que resultan en una vista de PDP.

* is_pdp_tvd: Calcula la proporción de "is_pdp" entre "total_visits_domain", representando la proporción de visitas a dominios que resultan en una vista de PDP.

Estas columnas creadas son derivadas de columnas ya existentes en el dataset original. Estas pueden ser útiles para analizar y categorizar artículos en función de su precio, descuento, longitud del título, patrones de visita y dominancia en el dominio.

#### Encoding de atributos categóricos

Además, aplicamos One Hot Encoding (OHE) a la columna "category_id" para crear representaciones binarias de los datos de categoría y Label Encoding a las columnas "platform", "product_id", "domain_id" y "logistic_type" para convertir los datos categóricos en valores numéricos en nuevas columnas:

* platform (LabelEncoded): Si el indicador lab_enc es True, esta columna se codifica con etiquetas para convertir los datos categóricos de la plataforma en valores enteros.

* product_id (LabelEncoded): Convierte la columna "product_id" en valores enteros.

* domain_id (LabelEncoded): Convierte la columna "domain_id" en valores enteros.

* logistic_type (LabelEncoded): Convierte la columna "logistic_type" en valores enteros.

* One-Hot Encoding (OHE) for category_id: Si el indicador de ohe es True, se aplica la codificación one-hot a la columna "category_id". Crea columnas binarias para cada categoría única y usa 1 para indicar la presencia de una categoría y 0 en caso contrario. El número de columnas depende del número de categorías únicas en el conjunto de datos.

* category_id_<category> (OHE): Columnas binarias creadas como resultado de la codificación one-hot para cada categoría única en la columna "category_id". **ESCRIBIR EJEMPLO** Inicialmente se hacía OHE por sobre todas las categorías exitentes, luego, para aliviar la demanda memoria RAM, se decidió hacer OHE por sobre las 10 categorías más populares y el resto simplemente contaba con todos 0.

#### Modificaciones de columnas existentes

Estas son algunas modificaciones y adiciones a las columnas que forman parte de los pasos de preprocesamiento de datos, que incluyen conversiones de tipo, manejo de valores faltantes y la creación de indicadores binarios para ciertas condiciones:

* free_shipping (Conversión de tipo): Convertimos la columna "free_shipping" a enteros. Probablemente representa una señal binaria donde 1 indica envío gratuito y 0 indica que no hay envío gratuito.

* fulfillment (Conversión de tipo): Convertimos esta columna a enteros. Esta columna probablemente representa un método de cumplimiento, y la conversión a números enteros puede usarse para codificar diferentes opciones de cumplimiento

* listing_type_id (Indicador Binario): La columna "listing_type_id" se modifica aplicando una función lambda. Si el valor es "gold_special," se establece en 0; de lo contrario, se establece en 1. Esto parece mapear diferentes tipos de listados a valores binarios para un análisis o modelado posterior. En otras palabras, se está creando una nueva columna que representa de manera binaria si un listado es "gold_special" (0) o cualquier otro tipo de listado (1) para su uso en análisis o modelos posteriores. En retrospectiva quizas podríamos haber invertido los valores de 0 y 1, ya que el tipo de listado "gold_special" es el más caro y por lo tanto el que más interesa a la empresa, pero en su momento consideramos que XGBoost iba a poder discernir en como manejar esta columna y no se lo volvió a considerar hasta el momento de escribir este informe.

#### Imputación de valores faltantes

* imp_user_id (Indicador Binario): Al igual que con "imp_is_pdp", esta columna se crea para indicar si la columna "user_id" tenía valores faltantes. Si "user_id" tiene valores faltantes, esta columna se establecerá en 1; de lo contrario, se establecerá en 0. Los valores faltantes en "user_id" se llenan con 0 y luego se convierten a números enteros.


* imp_is_pdp (Indicador Binario): Esta columna fue creada para indicar si la columna "is_pdp" tiene valores faltantes (NaN). Si "is_pdp" tiene un NaN, esta nueva columna se establecerá en 1; de lo contrario, se establecerá en 0. Además, los valores faltantes en "is_pdp" se llenan con 0 y luego se convierten a números enteros. Al crear esta columna se noto el primer "salto" en performance, ya que la columna is_pdp si es Falsa tiene correlación casi total con la columna a predecir ya que el usuario no pudo haber comprado el producto sin entrar a su página de detalles, algo que consideramos es quizas puede haber guardado en su carrito y luego en otra visita efectuar la compra desde allí. Al notar la importancia del dato, una cuestión que se implemento a lo largo de gran parte del desarrollo fue imputar los missings con predicciones de un modelo XGBoost entrenado con los datos que no tenían missings de los datos de Train. Sin embargo el modelo final no contaba con esta imputación.

En todos los modelos que entrenamos, la columna "is_pdp" fue la más importante, su ganancia rondaba siempre el rango 700-900, mientras que el resto de los atributos

#### Columna "tags"

Observamos que los Tags del producto podían contener información interesante sobre los productos por lo que  para cada etiqueta unica en la columna "tags", se crea una nueva columna con el nombre de la etiqueta. Estas nuevas columnas tienen valores booleanos (1 si el artículo contiene la etiqueta, 0 en caso contrario). Los nombres de estas columnas se basan en las etiquetas únicas encontradas en el conjunto de datos.

Luego en otra columna llamada "tag_counts" se almacena la cantidad de etiquetas asociadas a cada artículo. Calcula la cantidad de etiquetas dividiendo la columna "tags" y contando los elementos.

Una vez creadas las nuevas columnas relacionadas con las etiquetas, se elimina la columna original "tags" del DataFrame.

Notamos que 

##### Nuevas columnas con valores al cuadrado:

* Por cada atributo especificado en la lista poly_attrs (por ejemplo, "print_position," "offset," "discount," "price," "health," "original_price"), se crea una nueva columna elevando al cuadrado los valores de ese atributo. Los nombres de las columnas siguen el patrón <attribute_name>2, donde <attribute_name> es el nombre del atributo original. 

* Interaccion de columnas con suma de cuadrados: Para cada par de atributos especificados en la lista de polly_attrs, la combinacion de dos atributos tambien se crea (por ejemplo, "print_position2 + offset2"). Estas nuevas columnas representan la suma de cuadrados de los atributos correspondientes. El codigo esencialmente realiza la ingenieria de caracteristicas creando nuevas columnas con valores al cuadrado de los atributos especificados y tambien genera caracteristicas de interacción sumando los cuadrados de pares de atributos. Estas nuevas caracteristicas pueden capturar relaciones no lineales entre los atributos y pueden ser utiles para modelos de aprendizaje automático que se benefician de términos polinomiales o de interaccion.

##### Word2Vec y Reduccion de dimensionalidad con PCA:

tokenized_title: Esta columna se crea tokenizando la columna "title". Primero usa sent_tokenize para dividir el texto en oraciones, luego word_tokenize para dividir las oraciones en tokens, elimina los caracteres no alfabéticos y convierte las palabras en minúsculas. El resultado es una lista de oraciones tokenizadas.


Entrenamiento y carga del modelo Word2Vec (condicional):

* Si RETRAIN_W2C es True, se entrena un modelo Word2Vec utilizando los títulos tokenizados. El modelo se configura con hiperparámetros específicos como el tamaño del vector (300 embedings), el tamaño de la ventana, el recuento mínimo de palabras y otros. El modelo se guarda como "title_w2c.model"

* Si RETRAIN_W2C es False, se carga un modelo Word2Vec preentrenado del archivo "title_w2c.model". Esto le permite utilizar un modelo Word2Vec preexistente sin volver a entrenar.

title_embs: Esta columna contiene el promedio de todos los embeddings (representaciones vectoriales) de los títulos tokenizados. Nuestra intepretación es la media de los embeddings representa la estructura semantica de los titulos. Otra opción posible podría haber sido sumar todos los embeddings o hasta hacer el _dot product_ entre ellos.

Columnas de incrustación: Se crea una serie de columnas con nombres: "embeddings_i," $\forall i \in [0, n)$ siendo $n$ la cantidad de dimensiones de un embedding para que cada muestra contenga la representación vectorial de su título. 

En cuanto a la reducción de dimensionalidad, optamos utilizar PaCMAP con PCA para reducir la dimensionalidad de los embeddings de palabras. Esto nos permitió por un lado entrenar los modelos mas rápido, a la larga notamos que era a cuestas de la performance, por lo que se utilizó para poder evaluar otros atributos mas rapido, pero en el modelo final se utilizó el embedding de 300 dimensiones.
Para esto se crearon nuevas columnas con dimension reducida: Las nuevas columna con las dimensiones reducidas se llaman "pacmap_0," "pacmap_1," y así sucesivamente hasta el número de dimensiones especificado (dims) y se borran las columnas "embeddings_0," "embeddings_1," etc. originalmente creadas.

Tanto para cuando hacemos la reducción de dimensionalidad como cuando no, eliminamos las columnas "title_embs" y "tokenized_title" ya que no se utilizan en el modelo final.

##### Garantías:

Observamos que la columna "warranty" no estaba en un formato consistente. Algunos artículos tenían "Sin garantía" en el texto, mientras que otros tenían "Sin garantía" o "Sin garantía", probablemente por ser textos que fueron ingresados manualmente. Además, algunos artículos tenían "Vendedor" o "Fábrica" en el texto de la garantía, mientras que otros no. Para volver los datos mas uniformes se somete a un preprocesamiento, modificando la columna "warranty" original y creando nuevas variables representando distintosa aspectos de la garantia de cada artículo de la siguiente forma:

- Todo el texto se convierte a minúsculas.
- Los caracteres acentuados como "á" e "í" se reemplazan por sus equivalentes sin acentos.

Se crean las columnas: 

* warranty_saler: Esta columna se crea como una variable booleana (0 o 1) que indica si el texto de "warranty" contiene la palabra "vendedor" (garantía del vendedor). Es de tipo flotante, donde 1.0 indica la presencia de "vendedor" y 0.0 indica lo contrario.

* warranty_factory: Similar a "warranty_saler", esta columna se crea como una variable booleana que indica si el texto de "warranty" contiene la palabra "fábrica" (garantía de fábrica).

* warranty_no: Se crea otra variable booleana para indicar si el texto de "warranty" contiene "sin garantía" (no hay garantía).

* warranty_missing: Esta columna se crea como una variable booleana (0 o 1) que indica si el texto de "warranty" está ausente o no (NaNs).

* warranty_days: Esta columna extrae valores numéricos del texto de "warranty", específicamente buscando patrones como "X días", "X meses" o "X años". Calcula la duración en días en base a estos patrones.

* warranty_days_missing: Similar a "warranty_missing", esta columna se crea como una variable booleana que indica si la columna "warranty_days" está ausente o no (NaN o no NaN).

Rellenando Valores Faltantes:

Las columnas "warranty_saler", "warranty_factory", "warranty_no" y "warranty_days" se llenan con -1 en las filas donde la información correspondiente falta (NaN).

Eliminación de la Columna "warranty":

La columna original "warranty" se elimina del DataFrame después de procesarla.

#### Fechas:

La columna "date" es convertida a un formato "datetime" usando pd.to_datetime. Esto sirve para poder capturar información sobre aspectos temporales de la misma, incluyendo día, mes, día de la semana y momento del día (mañana, tarde, noche). Estas pueden ser utilizadas para realizar análisis basados en el tiempo con el fin de capturar patrones temporales en los datos. 

* day: Esta columna se crea para extraer y almacenar el componente del día (día del mes) de la columna "date".
* month: Esta columna se crea para extraer y almacenar el componente del mes de la columna "date".
* dayofweek: Esta extrae y almacena el día de la semana (0 para lunes, 6 para domingo) de la columna "date".

Columnas de Momento del Día (morning, afternoon, night): 
Estas columnas se crean en función del componente "hour" (hora) de la columna "date".

* morning: se establece en 1 si la hora está entre las 6 AM (inclusive) y las 12 PM (exclusivas).
* afternoon: se establece en 1 si la hora está entre las 12 PM (inclusive) y las 6 PM (exclusivas).
* night: se establece en 1 si la hora está entre las 6 PM (inclusive) y las 12 AM (medianoche, exclusiva).

Estas ultimas columnas no se utilizaron en el modelo final.

### Armado de conjunto de validación

Para probar todo esto, lo checkeabamos con 5 folds sobre el 80% de train, si mejoraba, reentrenabamos con todo el 80% de train. Otra opción, que nuestra validación tenga la misma cantidad de samples que los items de eval. Inicialmente nuestro holdout set era 25/75, cuando planteamos esto lo cambiamos a 20/80. 

Inicialmente, planteamos dividir el Dataset en 2, un Set de Test y otro de Train, este holdout set era en proporción 25/75. Luego, para poder validar el modelo, separamos el 20% del set de Train para tener un sub_conjunto de validación para evaluar rápido como afectaba al modelo cada cambio hecho a los atributos.

Una cuestión que consideramos era el desbalance que había en el Dataset respecto a la variable objetivo, ya que en la mayoría de los samples no ocurria la conversión. Para solucionar esto, se planteo utilizar un método de submuestreo aleatorio para reducir la cantidad de muestras de la clase mayoritaria (no vendidos) a la cantidad de muestras de la clase minoritaria (vendidos). Esto se hizo para evitar que el modelo se sobreajuste a la clase mayoritaria y para mejorar el rendimiento del modelo en la clase minoritaria. Sin embargo no observamos menor performance en el modelo, por lo que no se utilizó.

Alternativamente, planteamos oversamplear la clase minoritaria, pero por los distintos splits que le haciamos a los datasets, sobre todo luego cuando utilizamos K-Fold Cross Validation, no queriamos correr el riesgo de Data Leakage y evitar la repetición de datos en los distintos folds. Por lo que no se utilizó.

Luego, decidimos observar cuantos samples tenía el set sin datos de conversión sobre el que ibamos a hacer la predicción a subir a Kaggle que era de $19k$, por lo que decidimos modificar el split de Train/Test a 80/20, ya que ese 20% de Test (36k) se acercaba mas al doble de los samples de evaluación y nos permitia tener mas datos para entrenar el modelo.

### Exploración de Atributos por sobre los modelos

Para estudiar el comportamiento de los atributos por sobre nuestros modelos predictivos planteamos la siguiente metodología:

- Se decidía una modificaciones a realizar sobre los atributos, por ejemplo, eliminar una columna, agregar una nueva, etc.
- Se entrenaba un modelo XGBoost de Clasificación Binaria con el 75% del total de datos de Train (75% de 80% del total) y se hacía K-Fold Cross Validation (K=5) con el otro 25% y se evaluaba su performance respecto a la métrica AUC contra los resultados de utilizar el mismo modelo previo a la modificación. Luego se evaluaba tambien por sobre el 20% de Test.
- Si la performance mejoraba en Cross-Val (usualmente observabamos que mejoraba en el 20% de Test mas que en el 25% de Validación):
    - Por un lado se observaba la Gain y  de cada atributo para ver como el atributo que agregamos o modificamos afectaba al modelo.
    - Si pertenecía al top 10 de atributos mas importantes se reentrenaba un modelo con solo esos 10 atributos y se evaluaba su performance respecto al modelo con todos los atributos, para observar como afectaba al modelo la eliminación de los atributos menos importantes respecto a la inclusíon del nuevo atributo.
    - Luego se entrenaba el modelo con el 80% de Train y se evaluaba su performance respecto a los modelo previo a la modificación sobre el 20% de Test.

Esto lo repetimos para cada modificación que realizamos sobre los atributos.

Luego de este proceso, decidiamos si incluir esta modificación en la evaluación sobre los modelos de Ensamble (explicado luego) mas complejos que demoran mas en entrenar y no siempre podíamos estar reentrenando con distintas modificaciones.
    

### Otros modelos

#### DecisionTrees y RandomForest Classifiers

Inicialmente se probaron modelos de DecisionTrees y RandomForest. Los DecisionTrees se utilizaron solo para hacer un submit inicial con optimización de HyperOpt pero luego se descartaron. Aunque los Random Forest por si solos no llegaban a tener la performance de XGBoost (~0.83 vs  +0.88 en el resultado final), y eran mas lentos para entrenar que XGB cuando los atributos crecen (para 300 embeddings de Title era extremadamente mas lento que XGB), se optaron por utilizarlos en los modelos de Ensamble junto a XGBoost y los siguientes modelos.

#### AdaBoost Classifier

Este clasificador nos resulto en performance ROC_AUC mejor que RandomForest acercandose a XGBoost, pero tomaba casi el doble de tiempo en entrenar que XGB (2' ABC vs 1'22" XGB).

#### HistGradientBoosting Classifiers

Este clasificador nos resulto en performance ROC_AUC mejor que RandomForest los dos modelos anteriores y comparable con XGBoost y por sobre todo, el entrenamiento era mas rápido que XGB (16 segundos HGB vs 1'22" XGB).


### Modelos de Ensamble

Inicialmente para reducir el bias de nuestros modelos hicimos un ensamble promediando las probas predichas por los 4 modelos anteriores, de por si tuvo mejor performance en nuestra validación holdout pero no en el leaderboard, para ello se construyó la clase AverageClassifier que permitia a partir de modelos ya entrenados, promediar las probabilidades predichas por cada uno de ellos. Para poder entrenar los modelos en simultaneo y "compartir" sus predicciones sobre sus entrenamientos, se utilizó la clase VotingClassifier de sklearn. Luego entonces contabamos con un VotingClassifier compuesto por los modelos, lo llamamos Voting(XGB, ABC, HGB, RFC) que nos daba mejor performance que XGB en nuestra validación holdout y tambien en el leaderboard.

Ademas construimos un modelo que a partir de las predicciones de los modelos anteriores, entrenaba un modelo (hicimos uno con XGBoost y uno con LogisticRegression) con las predicciones de los modelos anteriores como atributos y los mismos labels a predecir, para ello utilizamos la clase StackingClassifier de sklearn, lo llamamos Stack(XGB, ABC, HGB, RFC). El modelo que mejor performance obtuvo en el Leaderboard Público fue un Stack(XGB, ABC, HGB) con un XGBoost como modelo final.
Armamos tambien otro StackingClassifier (Stack2) sin RFC para acortar en tiempo de entrenamiento pero su performance fue peor.

Otra opción que contemplamos fue promediar las predicciones de los modelos anteriores (Voting(XGB, ABC, HGB, RFC) y Stack(XGB, ABC, HGB, RFC) -> RegLog), a posteriori, este fue el modelo que mejor performance obtuvo en el Leaderboard Privado y fue de los mejores en el Público.

Finalmente, la opción por la que optamos fue hacer Bootstrap Aggregating (Bagging) por sobre XGBoost, para ello utilizamos la clase BaggingClassifier de sklearn, y _baggeamos_ 10 estimadores de XGBoost (Mas sobre esta decisión luego), lo llamamos Bagging(XGB). Este fue el modelo que mas tiempo toma en entrenar, ademas fue el que mejor performance obtuvo en nuestra Validación, pero no en el Leaderboard.

### Metodología Búsqueda de Hiperparámetros

En las primeras instancias del desarrollo, optamos por utilizar la librería HyperOpt para el tuneo de Hiperparametros. Lo utilizamos sobre DecisionTrees, RandomForest y XGBoost. 

El espació de hiperparametros para XGBoost que nos hayó la mejor performance sobre nuestra validación en su momento fue:
```
    "max_depth": hp.choice("max_depth", [2, 4, 8, 16, 32, 64, 128, None]),
    "learning_rate": hp.uniform("learning_rate", 0.01, 0.2),
    "n_estimators": hp.choice("n_estimators", [10, 50, 100, 200, 500]),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1),
    "gamma": hp.uniform("gamma", 0, 1),
    "min_child_weight": hp.choice("min_child_weight", [2, 4, 8, 16, 32, 64, 128]),
    "subsample": hp.uniform("subsample", 0.5, 1)
```
Esto podía correr en nuestra máquina por que todavía no contabamos con gran cantidad de atributos (apenas 55).

Sin embargo aquel que mejores resultados obtenía en el Leaderboard era aquel que utilizaba XGBoost con los hiperparametros default, por lo que decidimos replantear la búsqueda de hiperparametros.

En las siguientes instancias del desarrollo, optamos por utilizar la librería RandomizedSearchCV para el tuneo de Hiperparametros, por dos motivos principales: Hiperopt es muy lento en nuestras máquinas, y aunque corrimos todo parelelizado (8 cores de la CPU), luego debimos aprovechar del tiempo de computo de Google Colab sobre una GPU T4 para poder no solo paralelizar si no que entrenar XGBoost con la GPU y tambien HyperOpt, sin embargo Colab cuenta con menos RAM por lo que trabajar con mayor cantidad de atributos sobre Colab fue imposible; a mayor cantidad de atributos, la demanda de memoria RAM se disparó (la primera vez que observamos esto fue al hacer OHE sobre Category_ID) y no pudimos seguir trabajando sobre Colab. Para solucionar ambos problemas decidimos utilizar RandomizedSearchCV que nos permitia explorar de forma mas amplia el espacio de hiperparametros.

El espacio de hiperparametros que utilizamos sobre RandomizedSearchCV fue:

```
    "max_depth": [5, 10, 15, 30, 50],
    "n_estimators": [25, 50, 75, 100, 250, 500],
    "learning_rate": [0.001, 0.01, 0.1, 0.2],
    "colsample_bytree": [0.5, 0.6, 0.7, 0.8, 0.9],
    "gamma": [0, 0.1, 0.2, 0.3, 0.4, 0.5],
    "min_child_weight": [0, 1, 2, 3, 4, 5, 6],
    "subsample": [0.5, 0.6, 0.7, 0.8, 0.9],
```

Por cuestiones de tiempo, comenzamos a hacer RandomizedSearchCV con 100 fittings pero lo interrumpimos luego de haber completado 30'. Los hiperparametros que hayamos fueron los siguientes:

```
{'subsample': 0.8,
  'n_estimators': 500,
  'min_child_weight': 5,
  'max_depth': 5,
  'learning_rate': 0.1,
  'gamma': 0.1,
  'colsample_bytree': 0.7}
```

La busqueda de hiperparametros random la evaluamos sobre un 20% sobre el 80% de Train, esto nos resulto en una performance en nuestra validación de: $0.8923$

Con estos hiperparametros reentrenamos con los 80% de train enteros y obtivimos un ROC_AUC de $0.90084$

Al Baggear 10 estimadores de XGB sin el tuneo de hiperparametros, obtuvimos un ROC_AUC de $0.90156$ con un tiempo de entrenamiento de 24' en nuestra CPU, mientras al baggear 10 estimadores de XGB con los hiperparametros hayados obtuvimos un ROC_AUC de $0.9025$ con un tiempo de entrenamiento de 1h. Nos hubiera gustado considerar la cantidad de estimadores a baggear como parte de la optimización de hiperparametros pero no llegamos a hacerlo.

Otra opción que contemplamos, fue tunear los hiperparametros para el modelo Stack(XGB, AGB, HGB) -> XGB, tambien con RSCV, el espacio de hiperparametros era muy grande:

```
    "final_estimator__n_estimators": [25, 50, 100],
    "final_estimator__max_depth": [5, 10, 25, 50, 100],
    "final_estimator__learning_rate": [0.001, 0.01, 0.1, 0.2, 0.3],
    "final_estimator__gamma": [0, 0.1, 0.2, 0.3, 0.4, 0.5],
    "final_estimator__min_child_weight": [0, 1, 2, 3, 4, 5, 6],
    "final_estimator__subsample": [0.5, 0.6, 0.7, 0.8, 0.9],
    "final_estimator__colsample_bytree": [0.5, 0.6, 0.7, 0.8, 0.9],
    "xgb__n_estimators": [25, 50, 100],
    "xgb__max_depth": [5, 10, 25, 50, 100],
    "xgb__learning_rate": [0.001, 0.01, 0.1, 0.2, 0.3],
    "xgb__gamma": [0, 0.1, 0.2, 0.3, 0.4, 0.5],
    "xgb__min_child_weight": [0, 1, 2, 3, 4, 5, 6],
    "xgb__subsample": [0.5, 0.6, 0.7, 0.8, 0.9],
    "xgb__colsample_bytree": [0.5, 0.6, 0.7, 0.8, 0.9],
    "hgb__max_depth": [2, 3, 4, 5, 6, 7, 8, 9, 10],
    "hgb__learning_rate": [0.001, 0.01, 0.1, 0.2, 0.3],
    "hgb__max_iter": [100, 200, 300, 400, 500],
    "hgb__max_leaf_nodes": [10, 20, 30, 40, 50],
    "hgb__min_samples_leaf": [10, 20, 30, 40, 50],
    "hgb__l2_regularization": [0, 0.1, 0.2, 0.3, 0.4, 0.5],
    "abc_adaboostclassifier_n_estimators": [25, 50, 100],
    "abc_adaboostclassifier_learning_rate": [0.001, 0.01, 0.1, 0.2, 0.3],
```

No llegamos hacer una busqueda muy exhaustiva de hiperparametros, por lo que finalmente descartamos los stacks ya que no lo ibamos a poder hacer un buen Tune de sus hiperparametros, lo mismo para los modelos de ensamble de tipo Votings.

### Importancia de atributos

In [ ]:
### Breve 

### Responder: Dada una persona que se encuentra diseñando un anuncio de venta de un producto para publicar en este destacado retailer online, ¿en qué aspectos le recomendarían enfocarse? ¿Ven alguna debilidad en este análisis?